#Week 10 Group 40

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import random

## Load the data

In [2]:
ds = tfds.load("tiny_shakespeare",) ["train"]

Shuffling and writing examples to /root/tensorflow_datasets/tiny_shakespeare/1.0.0.incompleteEEH37M/tiny_shakespeare-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/tiny_shakespeare/1.0.0.incompleteEEH37M/tiny_shakespeare-validation.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/tiny_shakespeare/1.0.0.incompleteEEH37M/tiny_shakespeare-test.tfrecord


Dataset tiny_shakespeare downloaded and prepared to /root/tensorflow_datasets/tiny_shakespeare/1.0.0. Subsequent calls will reuse this data.


In [3]:
for i in ds:
  print(i["text"].numpy())

b"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger

##Preprocess the Data

In [4]:
  def change_backslash(str):
    symbols = "[:|,|.|?|!|;|']"
    string = tf.strings.lower(str)
    string = tf.strings.regex_replace(string, pattern = "\n\n", rewrite = " ")
    string = tf.strings.regex_replace(string, pattern = "\n", rewrite = " ")
    string = tf.strings.regex_replace(string, pattern = symbols, rewrite = "")
    return string

In [5]:
ds = tfds.load("tiny_shakespeare",) ["train"]
#extract the string from the dataset
for e in ds:
  string = e['text'].numpy()
  # Delete unwanted symbols
  string = change_backslash(string)

  string = tf.strings.split(string, sep = " ")
  np_arr = string.numpy()


In [6]:

print(np_arr.shape)
# get all unique words in the text
vals = np.unique(np_arr)#, return_index= True
#setup our dictionaries for words with corresponding embedding ID
dic_text_as_key = dict(zip(vals, range(len(vals))))
dic_index_as_key = dict(zip(dic_text_as_key.values(), dic_text_as_key.keys()))
print(len(dic_index_as_key))
print(dic_text_as_key[b"come"])
print(dic_index_as_key)

(182519,)
12269
2000
{0: b'', 1: b'&c', 2: b'--thou', 3: b'--what', 4: b'3', 5: b'a', 6: b'a-bed', 7: b'a-bleeding', 8: b'a-day', 9: b'a-doing', 10: b'a-dying', 11: b'a-fire', 12: b'a-growing', 13: b'a-high', 14: b'a-weary', 15: b'abandond', 16: b'abase', 17: b'abate', 18: b'abated', 19: b'abbey', 20: b'abbot', 21: b'abed', 22: b'abels', 23: b'abet', 24: b'abhor', 25: b'abhorrd', 26: b'abhorred', 27: b'abhorring', 28: b'abhors', 29: b'abhorson', 30: b'abide', 31: b'abides', 32: b'abilities', 33: b'ability', 34: b'abilitys', 35: b'abject', 36: b'abjects', 37: b'abjured', 38: b'able', 39: b'aboard', 40: b'abode', 41: b'abodements', 42: b'aboding', 43: b'abominable', 44: b'abortive', 45: b'abound', 46: b'aboundst', 47: b'about', 48: b'above', 49: b'abraham', 50: b'abrahams', 51: b'abreast', 52: b'abroach', 53: b'abroad', 54: b'abroad--', 55: b'absence', 56: b'absent', 57: b'absolute', 58: b'absolutely', 59: b'absolved', 60: b'abstains', 61: b'abstinence', 62: b'abstract', 63: b'abundance'

##Generate our Dataset

In [7]:
class gen_class(object):
    """
    Generates input target Pairs for words of the specified text.
    For a selected input word, returns tuples of the input-word and target-words in the scope of window sorrounding it.
    Parameters: text --> String Tensor to be Iterated over
                window --> The number of input/target pairs to create per input word
    """
    def __init__(self, text, window = 4):
        self.window = int(window/2)
        self.text = text
        self.i = self.window
        self.sequence_to_choose_from = range(1,10)
        self.seq = []

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()

    def next(self):
        while self.i < len(self.text)-self.window:
          ls_inp = []
          ls_tar = []
          for idx in range(-self.window, self.window +1):
            target = self.text[self.i]
            if idx != 0:
              #ls.append((self.text[self.i], (self.text[int(self.i+idx)])))
              ls_inp.append(self.text[self.i])
              ls_tar.append(self.text[int(self.i+idx)])
          self.i += 1
          return ls_inp, ls_tar# ls
window = 4
#ds = tf.data.Dataset.from_generator(generator=gen_class ,output_shapes = ([window,2]), output_types=(tf.dtypes.string), args= [string, window])
ds = tf.data.Dataset.from_generator(generator=gen_class ,output_shapes = ([2,window]), output_types=(tf.dtypes.string), args= [string, window])
ds = (ds.cache().shuffle(buffer_size = 10000, reshuffle_each_iteration=True).batch(64, )).prefetch(tf.data.experimental.AUTOTUNE)
for i in ds.take(1):
  print(i)

tf.Tensor(
[[[b'one' b'one' b'one' b'one']
  [b'country' b'than' b'voluptuously' b'surfeit']]

 [[b'in' b'in' b'in' b'in']
  [b'you' b'must' b'no' b'way']]

 [[b'words' b'words' b'words' b'words']
  [b'give' b'good' b'to' b'thee']]

 [[b'which' b'which' b'which' b'which']
  [b'show' b'themselves' b'in' b'the']]

 [[b'pity' b'pity' b'pity' b'pity']
  [b'oerwhelmd' b'my' b'i' b'request']]

 [[b'the' b'the' b'the' b'the']
  [b'so' b'now' b'gates' b'are']]

 [[b'no' b'no' b'no' b'no']
  [b'pray' b'now' b'more' b'my']]

 [[b'speak--it' b'speak--it' b'speak--it' b'speak--it']
  [b'well' b'as' b'tauntingly' b'replied']]

 [[b'being' b'being' b'being' b'being']
  [b'which' b'we' b'members' b'should']]

 [[b'tinder-like' b'tinder-like' b'tinder-like' b'tinder-like']
  [b'hasty' b'and' b'upon' b'too']]

 [[b'a' b'a' b'a' b'a']
  [b'sicinius' b'such' b'nature' b'tickled']]

 [[b'them' b'them' b'them' b'them']
  [b'hath' b'held' b'that' b'tos']]

 [[b'store-house' b'store-house' b'store-house' b's

In [8]:
# USE FOR DICT SORTED BY OCCURENCE
from collections import defaultdict

D = defaultdict(int)
for word in np.unique(np_arr):
    D[word] += 1
print(len(D))
D1 = dict(zip(D.values(),D.keys()))
print(D1)
d = dict(zip(sorted(D1),[D1[key] for key in sorted(D1)]))
print(d)
#for key in sorted(D1):
#  print( "%s : %s"% (key, D1[key]))

12269
{1: b'zounds'}
{1: b'zounds'}


##Model

In [9]:
def train_step(model, input_batch, target_batch, loss_function, optimizer, training = True):
  with tf.GradientTape() as tape:
    for input,target in zip(input_batch,target_batch):
      print(f"input in trainstep:{input}")
      print(f"target in trainstep:{target}")
      loss = model(input, target)
    # hs = tf.zeros([input.shape[0],input.shape[2]])
    # expectation = model(input, hs, training)        
    # loss = loss_function(target, expectation)
    # gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  train_accuracy = tf.cast(tf.argmax(target, axis = 1) == tf.argmax(expectation, axis = 1), tf.int16)
  return loss, train_accuracy

In [10]:
def train_model(model = None, num_epochs = 25, optimizer = None, learning_rate = 0.0005, loss_function = tf.keras.losses.CategoricalCrossentropy()):
  num_epochs = num_epochs
  learning_rate = learning_rate
  running_average_factor = 0.95
  # use categorical crossentropy as loss function
  loss_function = loss_function
  # use adam as optimizer
  if optimizer is None:
    optimizer =  tf.keras.losses.CategoricalCrossentropy(learning_rate)
  else:
    optimizer = optimizer
  # initialize the model
  model = model

  # lists for plotting later
  train_loss_list = []
  train_accuracy_list = []

  # we train for the predefined number of epochs
  for step in range(num_epochs):
    # reset average for each step
    changing_average_loss = 0
    # train the model with the training data
    for batch in ds:
      input_batch = batch[:,0,:]
      target_batch = batch[:,1,:]
      #print("input batch :")
      #print(input_batch)
      #print("target batch :")
      #print(target_batch)
      #for input,target in zip(sample[:,0,:],sample[:,1,:]):
        # print(input)
        # print("split")
        # print(target)
      # perform training step and store the loss and accuracy of step
      train_loss, train_accuracy = train_step(model, input_batch, target_batch, loss_function, optimizer)
      changing_average_loss = running_average_factor * changing_average_loss + (1 - running_average_factor) * train_loss
    train_loss_list.append(changing_average_loss.numpy())
    train_accuracy_list.append(np.mean(train_accuracy))
    
    # print values during training
    print("Epoch: " + str(step + 1))
    print("Training Accuracy: " + str(train_accuracy_list[step]))
    print("Train Loss: " + str(train_loss_list[step]))
    print()

In [11]:
class SkipGram(tf.keras.layers.Layer):
  def __init__(self, vocabulary, embedding_size = 100):
    super(SkipGram, self).__init__()
    self.vocab = vocabulary
    self.vocab_len = len(self.vocab)
    #self.embedding_size = embedding_size
    self.embedding_matrix = tf.keras.layers.Layer.add_weight(self, shape = [self.vocab_len,embedding_size], dtype = tf.int64)
    self.score_matrix = tf.keras.layers.Layer.add_weight(self, shape = [self.vocab_len,embedding_size], dtype = tf.int64)
    self.score_matrix_bias = tf.keras.layers.Layer.add_weight(self, shape = [self.vocab_len], dtype = tf.int64)
    self.loss = tf.nn.nce_loss
    print(f"embedding_matrix in init:{self.embedding_matrix}")
    print(f"score_matrix in init:{self.score_matrix}")

  def __call__(self, input, target):
    #print(f"input in call:{input}")
    #print(f"target in call:{target}")
    
    input = tf.one_hot([self.vocab[i.numpy()] for i in input],len(self.vocab),dtype = tf.int64)
    target = tf.one_hot([self.vocab[i.numpy()] for i in target],len(self.vocab),dtype = tf.int64)#tf.Variable([self.vocab[i.numpy()] for i in target], dtype = tf.dtypes.int64, )#
    print(f"input shape in call(onehot):{input.shape}")
    target = np.argmax(target, axis = 1)
    print(f"target shape in call(onehot):{target.shape}")
    #print(f"target shape in call(onehot):{np.argmax(target, axis = 1)}")
    embedding = tf.nn.embedding_lookup(self.embedding_matrix, np.argmax(input, axis = 1))
    print(f"shape of embedding for loss: {embedding.shape}")
    print(f"type of embedding for loss: {embedding.dtype}")
    print(f"shape of score-matrix for loss: {self.score_matrix.shape}")
    target = tf.reshape(target, (-1,1))
    print(f"shape of target for loss: {self.score_matrix.dtype}")
    print(f"shape of target for loss: {self.score_matrix_bias.dtype}")
    print(f"shape of target for loss: {target.dtype}")
    print(f"shape of target for loss: {embedding.dtype}")
    loss = tf.nn.nce_loss(self.score_matrix, self.score_matrix_bias, target, embedding, 10, self.vocab_len, 1,  )
    print(loss)
    return loss

In [12]:
nn = SkipGram(vocabulary= dic_text_as_key )#dic_text_as_key)
train_model(nn, 1)

embedding_matrix in init:<tf.Variable 'Variable:0' shape=(12269, 100) dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>
score_matrix in init:<tf.Variable 'Variable:0' shape=(12269, 100) dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>
input in trainstep:[b'an-hungry' b'an-hungry' b'an-hungry' b'an-hungry']
target in trainstep:[b'they' b'were' b'sighd' b'forth']
input shape in call(onehot):(4, 12269)
target shape in call(onehot):(4,)
shape of embedding for loss: (4, 100)
type of embedding for loss: <dtype: 'int64'>
shape of score-matrix for loss: (12269, 100)
shape of target for loss: <dtype: 'int64'>
shape of target for loss: <dtype: 'int6

InvalidArgumentError: ignored

## Preprocess the data

In [ ]:
# # all to lower case
# # remove symbols etc.
# # create vocabulary: generally just all words as vectors with words with similar context as similar vectors



# def preprocess(input, label):
#     input = input/255
#     input = tf.reshape(input,(28,28,1))
#     print(input)
#     label = tf.one_hot(label, 10)
#     label = tf.reshape(label, (-1,))
#     return (input, label)
# #print(input_train.shape)
# #print(label_train.shape)
# #print(input_test.shape)
# train_images = tf.data.Dataset.from_tensor_slices(train_images)
# train_labels = tf.data.Dataset.from_tensor_slices(train_labels)
# test_images = tf.data.Dataset.from_tensor_slices(test_images)
# test_labels = tf.data.Dataset.from_tensor_slices(test_labels)
# train_ds = tf.data.Dataset.zip((train_images, train_labels)) 
# test_ds = tf.data.Dataset.zip((test_images, test_labels))
# train_ds = train_ds.map(preprocess)
# test_ds = test_ds.map(preprocess)
# train_ds =   (tf.data.Dataset.shuffle(train_ds,buffer_size = 100000, reshuffle_each_iteration=True).batch(64)).prefetch(3)
# test_ds =   (tf.data.Dataset.shuffle(test_ds,buffer_size = 100000, reshuffle_each_iteration=True).batch(64)).prefetch(3)

In [ ]:
# class gen_class(object):
#     """
#     Generates input target Pairs for words of the specified text.
#     For a selected input word, returns tuples of the input-word and target-words in the scope of window sorrounding it.
#     Parameters: text --> String Tensor to be Iterated over
#                 window --> The number of input/target pairs to create per input word
#     """
#     def __init__(self, text, window = 4):
#         self.window = int(window/2)
#         self.text = text
#         self.i = self.window
#         self.sequence_to_choose_from = range(1,10)
#         self.seq = []

#     def __iter__(self):
#         return self

#     def __next__(self):
#         return self.next()

#     def next(self):
#         while self.i < len(self.text)-self.window:
#           ls_inp = []
#           ls_tar = []
#           for idx in range(-self.window, self.window +1):
#             target = self.text[self.i]
#             if idx != 0:
#               #ls.append((self.text[self.i], (self.text[int(self.i+idx)])))
#               ls_inp.append(self.text[self.i])
#               ls_tar.append(self.text[int(self.i+idx)])
#           self.i += 1
#           return ls_inp, ls_tar# ls
# window = 4
# #ds = tf.data.Dataset.from_generator(generator=gen_class ,output_shapes = ([window,2]), output_types=(tf.dtypes.string), args= [string, window])
# ds = tf.data.Dataset.from_generator(generator=gen_class ,output_shapes = ([2,window]), output_types=(tf.dtypes.string), args= [string, window])
# ds = (ds.cache().shuffle(buffer_size = 10000, reshuffle_each_iteration=True).batch(64, )).prefetch(tf.data.experimental.AUTOTUNE)
# for i in ds.take(1):
#   print(i[0,1])
#   print(i[0,1,1])

# class SkipGram(tf.keras.layers.Layer):
#   def __init__(self, vocabulary, embedding_size = 64):
#     super(SkipGram, self).__init__()
#     self.vocab = vocabulary
#     #self.embedding_size = embedding_size
#     self.embedding_matrix = tf.keras.layers.Layer.add_weight(self, shape = [len(self.vocab),embedding_size])
#     self.loss = tf.nn.nce_loss

#   def __call__(self, input, target):
#     # for e in input:
#     #   input = tf.one_hot([self.vocab[i.numpy()] for i in e],len(self.vocab),dtype = tf.dtypes.int32)
#     #   #print(input)
#     #input = [tf.one_hot([self.vocab[i.numpy()] for i in e],len(self.vocab),dtype = tf.dtypes.int32) for e in input]
#     input = tf.map_fn(fn=lambda t: tf.Variable(self.vocab[i] for i in t), elems = input)
#     print(input)
#     # for e in target:
#     #   target = tf.one_hot([self.vocab[i.numpy()] for i in e],len(self.vocab),dtype = tf.dtypes.int32)
#     #   print(target)
#     target = [tf.one_hot([self.vocab[i.numpy()] for i in e],len(self.vocab),dtype = tf.dtypes.int32) for e in target]
#     print(target)
#     embedding = tf.nn.embedding_lookup(self.embedding_matrix, input )
#     loss = self.loss(self.embedding_matrix, None, target, embedding, 10, len(self.vocab))
#     print(loss)
#     return loss

In [13]:
a = tf.zeros([3,2])
print(a)
print(a[:,0])

tf.Tensor(
[[0. 0.]
 [0. 0.]
 [0. 0.]], shape=(3, 2), dtype=float32)
tf.Tensor([0. 0. 0.], shape=(3,), dtype=float32)


## Preprocessing